In [ ]:
% load_ext dotenv
% dotenv

import os
import decimal
from datetime import datetime

import ipywidgets as widgets
import plotly.graph_objects as go
import pytz
import requests
import pandas as pd
from dateutil import tz
from ipywidgets import interact

token = os.getenv("TINKOFF_API_TOKEN")

def quotation_to_decimal(quotation):
    units = quotation['units']
    nano = str(quotation['nano'])

    if len(nano) > 8:
        return round(decimal.Decimal(units + '.' + nano), 2)
    else:
        return round(decimal.Decimal(units + '.' + '0' + nano), 2)


def get_all_etfs():
    response = requests.post(
        'https://invest-public-api.tinkoff.ru/rest/tinkoff.public.invest.api.contract.v1.InstrumentsService/Etfs',
        json={'instrumentStatus': 'INSTRUMENT_STATUS_BASE'},
        headers={
            'Authorization': 'Bearer ' + token,
            'accept': 'application/json', 'Content-Type': 'application/json'})
    return response.json()['instruments']


def get_price(figi):
    response = requests.post(
        'https://invest-public-api.tinkoff.ru/rest/tinkoff.public.invest.api.contract.v1.MarketDataService/GetLastPrices',
        json={'figi': [figi]},
        headers={
            'Authorization': 'Bearer ' + token,
            'accept': 'application/json', 'Content-Type': 'application/json'})

    return quotation_to_decimal(response.json()['lastPrices'][0]['price'])

def str_to_datetime_local(str_date_time):
    date_time = datetime.strptime(str_date_time+'+0000', '%Y-%m-%dT%H:%M:%SZ%z')
    return date_time.astimezone(tz.tzlocal())

def datetime_to_str(date_time):
    str_datetime = date_time.astimezone(pytz.utc).strftime('%Y-%m-%dT%H:%M:%S.000Z')
    return str_datetime

def moex_str_to_datetime_local(str_date_time):
    date_time = datetime.strptime(str_date_time+'+0300', '%Y-%m-%d %H:%M:%S%z')
    return date_time.astimezone(tz.tzlocal())

def datetime_to_moex_str(date_time):
    str_datetime = date_time.astimezone(pytz.timezone('Europe/Moscow')).strftime('%Y-%m-%d')
    return str_datetime


def get_candles(figi, date_start, date_end):
    response = requests.post(
        'https://invest-public-api.tinkoff.ru/rest/tinkoff.public.invest.api.contract.v1.MarketDataService/GetCandles',
        json={'figi': [figi], 'from': [datetime_to_str(date_start)], 'to': [datetime_to_str(date_end)], 'interval': ['CANDLE_INTERVAL_1_MIN']},
        headers={
            'Authorization': 'Bearer ' + token,
            'accept': 'application/json',
            'Content-Type': 'application/json'})
    df = pd.DataFrame(response.json()['candles'])
    df['open'] = df['open'].map(quotation_to_decimal)
    df['close'] = df['close'].map(quotation_to_decimal)
    df['low'] = df['low'].map(quotation_to_decimal)
    df['high'] = df['high'].map(quotation_to_decimal)
    df['time'] = df['time'].map(str_to_datetime_local)
    return df


def get_inav_prices_moex(ticker, date_start, date_end):

    df_inav = pd.DataFrame()

    start = 0
    while True:
        next_response = requests.get(
            url=f'https://iss.moex.com/iss/engines/stock/markets/index/boardgroups/104/securities/{ticker}A/candles.json',
            params={
                'interval': 1,
                'from': datetime_to_moex_str(date_start),
                'till': datetime_to_moex_str(date_end),
                'start': start
            })

        candles = next_response.json()['candles']['data']
        if len(candles) > 0:
            next_df = pd.DataFrame([{'time': moex_str_to_datetime_local(candle[6]), 'price': candle[0]} for candle in candles])
            df_inav = pd.concat([df_inav, next_df], ignore_index=True)
            start+=500
        else:
            break

    return df_inav

def draw_chart(figi, ticker, date_start, date_end):

    df_candles = get_candles(figi, date_start, date_end)

    fig = go.Figure(data=[go.Candlestick(x=df_candles['time'],
                                         open=df_candles['open'],
                                         high=df_candles['high'],
                                         low=df_candles['low'],
                                         close=df_candles['close'])
                                         ])
    fig.update_traces(name='market_data', selector=dict(type='candlestick'))
    df_inav_prices = get_inav_prices_moex(ticker, date_start, date_end)

    fig.add_trace(go.Scatter(x=df_inav_prices['time'], y=df_inav_prices['price'], name='inav'))

    fig.show()


In [2]:
tinkoff_etfs = filter(lambda etf: 'Тинькофф' in etf['name'], get_all_etfs())
dropdown_options = [(etf['name'] + ' (' + etf['ticker'] + ')', (etf['figi'], etf['ticker'])) for etf in tinkoff_etfs]

figi_widget = widgets.Dropdown(
    options=dropdown_options,
    description='Инструмент: ',
)
date_start_widget = widgets.DatetimePicker(
    description='Начало периода',
    disabled=False
)
date_end_widget = widgets.DatetimePicker(
    description='Конец периода',
    disabled=False
)

@interact(figi_and_ticker=figi_widget, date_start=date_start_widget, date_end=date_end_widget)
def on_select(figi_and_ticker, date_start, date_end):
    figi = figi_and_ticker[0]
    ticker = figi_and_ticker[1]
    return draw_chart(figi, ticker, date_start, date_end)

interactive(children=(Dropdown(description='Инструмент: ', options=(('Тинькофф iMOEX (TMOS)', ('BBG333333333',…